In [1]:
import h5py
import os
import cv2
import numpy as np
import scipy.spatial.distance
import scipy.io
import mat73

from skimage import io
from sklearn.cluster import KMeans
from pandas import DataFrame

In [4]:
File_path = '/home/cse/Desktop/Raj_M.Tech_Project/Raj/location_append_normalised_mat_7_bAND_MIN_MAX'
HS_files = os.listdir(File_path)
print("HS_files", HS_files)
save_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Test'

HS_files ['0016.mat', '0012.mat']


In [5]:
def create_dict(temp_dict, dict_size):
# dict_size = 10
  # temp_dict = {}
  for i in range(dict_size):
    temp_dict.update({i: list()})
  return temp_dict

In [6]:
number_of_clusters = 10

In [7]:
dict_size = 10
pixel_values = create_dict({}, dict_size)
cluster_means = create_dict({}, dict_size)
mean_list = []

In [8]:
print('pixel values', pixel_values)
print('cluster means', cluster_means)

pixel values {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}
cluster means {0: [], 1: [], 2: [], 3: [], 4: [], 5: [], 6: [], 7: [], 8: [], 9: []}


In [9]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [10]:
im_labelled_images_path = os.path.join(save_path, 'labelled_images')
mkdir(im_labelled_images_path)

In [12]:
for file in HS_files:
  input_img = os.path.join(File_path, file)
  # mat_dict = h5py.File(input_img)
  mat_dict = scipy.io.loadmat(input_img)
  for key, value in mat_dict.items():
    image_data = np.array(value)
    # print('Image data before axis move', image_data.shape)
    image_data = np.moveaxis(image_data, 0, 2)
    # print('Image data after axis move', image_data.shape)
    # image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    img = image_data.reshape((-1, 81))
    img = np.float32(img)
    # print('img shape', img.shape)
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.2)
    
    # k = 50

    ret, labels, centers = cv2.kmeans(img, number_of_clusters, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    # centers = np.uint8(centers)
    # print('labels shape before flattening', labels.shape)
    labels = labels.flatten()
    # print('labels shape after flattening', labels.shape)
    labels = labels.reshape(1024, 768)
    # print('labels after reshapping', labels.shape)
    # print('center shape', centers.shape)
    # segmented_image = centers[labels.flatten()]
    # segmented_image = segmented_image.reshape(image_data.shape)
    # print('segmented_image shape', segmented_image.shape)
    file = file.rsplit('.', 1)

    for i in range(labels.shape[0]):
      for j in range(labels.shape[1]):
        label = labels[i][j]
        pixel_values[label].append(image_data[i][j][:])

    for label in pixel_values:
      channel_mean = np.mean(pixel_values[label], axis = 0)
      cluster_means[label].append(channel_mean)
    
    labels = cv2.rotate(labels, cv2.ROTATE_90_CLOCKWISE)
    labels = cv2.flip(labels, 1)
    save_im_labelled_images = im_labelled_images_path + '/' + file[0] + '.png'
    cv2.imwrite(save_im_labelled_images, labels)

  mean_list.append(cluster_means)
  pixel_values = dict((k, list()) for k in pixel_values)
  cluster_means = dict((k, list()) for k in cluster_means)

print('!!!!!! Cluster Mean Calculated Successfully !!!!!!')

ValueError: Unknown mat file type, version 113, 198

In [ ]:
# print(mean_list)

In [ ]:
# print(labels)

In [ ]:
# for item in mean_list:
#   # print(item)
#   for val in item.values():
#     # print(val)
#     for ele in val:
#       if isinstance(ele, float):
#         val[0] = [0]

In [23]:
dist = 0
clus_dist = []
euc_dist = []
for i in range(len(mean_list)):
  for j in range(len(mean_list[i])):
    for k in range(len(mean_list[i])):
      # print('i-j', mean_list[i][j][0])
      # print('i-k', mean_list[i][k][0])
      dist += scipy.spatial.distance.euclidean(mean_list[i][j][:], mean_list[i][k][:])
      # dist += np.linalg.norm(mean_list[i][j][0] - mean_list[i][k][0])
    clus_dist.append(dist)
    dist = 0

  euc_dist.append(clus_dist)
  clus_dist = []

print('!!!!!! Euclidean Distance Calculated Successfully !!!!!!')

!!!!!! Euclidean Distance Calculated Successfully !!!!!!


In [24]:
# print(euc_dist)

In [25]:
# mean_list_ind = 0
euc_dist_ind = 0
clusters_ind = -1

File_path_labelled_images = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Test/labelled_images'
HS_files_labelled_images = sorted(os.listdir(File_path_labelled_images))
print("HS_files_labelled_images", HS_files_labelled_images)

save_path = '/home/cse/Desktop/Raj_M.Tech_Project/Approach4/HS_Results/Test'
im_final_segmented_images = os.path.join(save_path, 'Final_Images')
mkdir(im_final_segmented_images)

for labelled_images in HS_files_labelled_images:
  file = labelled_images.rsplit('.', 1)
  input_labelled_images = os.path.join(File_path_labelled_images, labelled_images)
  img = cv2.imread(input_labelled_images)
  # print('Image before channel removal', img.shape)
  img = np.max(img, axis = 2)
  # print('Image after channel removal', img.shape)
  # print('Image shape[0]', img.shape[0])
  # print('Image shape[1]', img.shape[1])
  
  clusters_ind += 1
  for clusters in range(clusters_ind, len(mean_list), 1):
    for label, mean in mean_list[clusters].items():
      if np.mean(mean[0]) != 0:
        for i in range(img.shape[0]):
          for j in range(img.shape[1]):
            # print(img[i][j])
            if label == img[i][j]:
              # print(1)
            #   print(euc_dist[clusters])
              img[i][j] = euc_dist[clusters][euc_dist_ind]
        euc_dist_ind += 1
        # print('If euc_dist_ind', euc_dist_ind)
      
      else:
        euc_dist_ind += 1
        # print('Else euc_dist_ind', euc_dist_ind)
        continue
    euc_dist_ind = 0
  
    # clusters += 1
    # img = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
    # img = cv2.flip(img, 1)
    
    max_pixel_val = np.max(img)
    img = img/max_pixel_val
    img = img * 255

    # kernel = np.ones((9, 9), np.uint8)
    # img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)

    save_im_final_featureMap = im_final_segmented_images + '/' + file[0] + '.png'
    cv2.imwrite(save_im_final_featureMap, img)
    break

  # save_im_fdict_size = 5inal_featureMap = im_final_segmented_images + '/' + file[0] + '.png'
  # cv2.imwrite(save_im_final_featureMap, img)

print('!!!!!!!!! Image saved successfully !!!!!!!!!!')

HS_files_labelled_images ['0041.png', '0042.png', '0043.png', '0044.png', '0045.png', '0046.png', '0047.png', '0050.png', '0051.png', '0056.png', '0061.png', '0062.png', '0064.png', '0065.png', '0070.png', '0071.png', '0072.png', '0076.png']
!!!!!!!!! Image saved successfully !!!!!!!!!!
